In [1]:
"""
Process Name : STEP 5 : CREATION OF MODEL
"""
#=======================================================================================================
### Required Imports ###
#============================================================================================================
try:
    import sys  # System-specific parameters and functions
    import logging # for logs
    from datetime import datetime  # Date and time
    import socket # network communication
    import warnings  # Warning control
    import os # Interacting with the operating system
    import pandas as pd  # Data manipulation
    import numpy as np  # Numerical operations
    from sklearn.utils import resample  # Resampling utilities
    from imblearn.over_sampling import SMOTE  # Oversampling for imbalanced data
    from sklearn.ensemble import RandomForestClassifier  # Random Forest classifier
    import lightgbm as lgb  # LightGBM for boosting
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Model evaluation metrics
    from sklearn.multioutput import MultiOutputClassifier  # Multi-output classifier wrapper
    from sklearn.model_selection import train_test_split  # Split dataset
    from sklearn.preprocessing import LabelEncoder  # Encode labels as integers
    from sklearn.cluster import KMeans  # K-means clustering
    from sklearn.preprocessing import StandardScaler  # Feature standardization
    import matplotlib.pyplot as plt  # Plotting
    from sklearn.preprocessing import MinMaxScaler  # Feature scaling
    from tensorflow.keras.models import Sequential  # Keras Sequential model
    from tensorflow.keras.callbacks import EarlyStopping  # Early stopping callback
    from sklearn.metrics import silhouette_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score  # Various metrics
    from sklearn.ensemble import RandomForestRegressor  # Random Forest regressor
    from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense  # CNN layers
    from joblib import dump, load #saving joblibs
except Exception as err:
    print("Exception raised while importing the packages")
    print(f'Exception: {err}')
    #input("press Enter to Close")
    sys.exit()

2024-09-05 15:44:49.983770: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#=========================================================================
### Initialization ###
#=========================================================================
try:
    path = os.getcwd()
    curr_time = datetime.now()

    ### Log Files declaration ###
    log_folder = os.path.join(path, 'Logs')
    log_date_fmt = str(curr_time.strftime('%Y')) + '-' + str(curr_time.strftime('%m')) + '-' + str(curr_time.strftime('%d')) + "_" + str(curr_time.strftime("%H")) + "-" + str(curr_time.strftime("%M"))

    audit_log_file = "Audit_creation_of_model.log"
    audit_log_file = os.path.join(log_folder, audit_log_file)
    
    error_log_file = "Error_creation_of_model.log"
    error_log_file = os.path.join(log_folder, error_log_file)

    ### Creating log folder ###
    if not os.path.exists(log_folder):
        os.makedirs(log_folder)
        
    ### Function: Logger setup ###
    def setup_logger(logger_name, log_file, level=logging.INFO):
        logger = logging.getLogger(logger_name)
        formatter = logging.Formatter(socket.gethostname()+' : '+'%(asctime)s : %(levelname)s : [%(filename)s:%(lineno)d] : %(message)s')

        fileHandler = logging.FileHandler(log_file, mode='w')

        fileHandler.setFormatter(formatter)

        streamHandler = logging.StreamHandler(sys.stdout)
        streamHandler.setFormatter(formatter)

        logger.setLevel(level)
        logger.addHandler(fileHandler)
        logger.addHandler(streamHandler)
        return logger
        
    ### Setting up the logger ###
    setup_logger('audit', audit_log_file, level=logging.INFO)
    setup_logger('error', error_log_file, level=logging.ERROR)

    audit_logger = logging.getLogger('audit')
    error_logger = logging.getLogger('error')
    audit_logger.info('Process start')

except Exception as err:
    print('Setting up the logger failed')
    print(f'Exception: {err}')
    #input("press Enter to Close")
    sys.exit()

192.168.1.112 : 2024-09-05 15:44:51,899 : INFO : [4183217348.py:45] : Process start


In [3]:
#==================================================================================================
### Ignore Warnings ### 
#==================================================================================================
try:
    audit_logger.info('Ignore Warnings')
    warnings.filterwarnings('ignore') ## Suppress all warnings
except Exception as err:
    audit_logger.info('Ignore Warnings - Failed')
    error_logger.error('Ignore Warnings - Failed')
    error_logger.error('Exception: ', exc_info=True)
    #input("press Enter to Close")
    sys.exit()

192.168.1.112 : 2024-09-05 15:44:51,903 : INFO : [2561458356.py:5] : Ignore Warnings


In [4]:
#==================================================================================================
### Create folder structure ### 
#==================================================================================================
try:
    audit_logger.info('Create folder structure')
    excel_folder = os.path.join(path, 'Excel_files')
    if not os.path.isdir(excel_folder):
        os.mkdir(excel_folder)
    saved_folder = os.path.join(path, 'Saved_files')
    if not os.path.isdir(saved_folder):
        os.mkdir(saved_folder)
    output_folder = os.path.join(path, 'Output')
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)
except Exception as err:
    audit_logger.info('Create folder structure - Failed')
    error_logger.error('Create folder structure - Failed')
    error_logger.error('Exception: ', exc_info=True)
    #input("press Enter to Close")
    sys.exit()

192.168.1.112 : 2024-09-05 15:44:51,909 : INFO : [4180990911.py:5] : Create folder structure


In [5]:
#==================================================================================================
### Cluster formation and applying smote ### 
#==================================================================================================
try:
    audit_logger.info('Cluster formation and applying smote and split')
    df = pd.read_excel(os.path.join(excel_folder, "preprocessed_data.xlsx"))
    eval_df = pd.read_excel(os.path.join(excel_folder, "evaluation_data_preprocessed.xlsx"))
    df = pd.concat([df, eval_df], ignore_index=True)

    data_for_cluster = df[['Gender', 'Weight_kg', 'Height_cm','Preferred_Clothing_Type', 'Clothing_Size',
                           'Favourite_Color_r', 'Favourite_Color_g', 'Favourite_Color_b','Best_Color_Confidence_r', 
                           'Best_Color_Confidence_g','Best_Color_Confidence_b', 'birthplace_lat','birthplace_lon', 
                           'resi_lat', 'resi_lon', 'Age', 'Personality']]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(data_for_cluster)
    
    dump(scaler, os.path.join(saved_folder, 'scaler_1.joblib')) #save the scaler to disk
    
    # Choose the optimal number of clusters
    k_optimal = 2
    kmeans = KMeans(n_clusters=k_optimal, random_state=42)
    kmeans.fit(X_scaled)
    
    # Save the model to disk
    dump(kmeans, os.path.join(saved_folder,'kmeans_model.joblib'))
    
    clusters = kmeans.predict(X_scaled)
    
    # Add the cluster labels to your original dataframe
    data_for_cluster['Cluster'] = clusters
    df['Cluster'] = clusters
    
    # Apply SMOTE
    sm = SMOTE(random_state=123)
    X, y = df.drop('Preferred_Clothing', axis=1), df['Preferred_Clothing']
    X_res, y_res = sm.fit_resample(X, y)
    
    # Combine resampled data back into a DataFrame
    df_resampled = pd.DataFrame(X_res, columns=X.columns)
    df_resampled['Preferred_Clothing'] = y_res
    
    df_resampled.drop_duplicates(inplace=True)
    df_resampled.fillna(0, inplace=True)
except Exception as err:
    audit_logger.info('Cluster formation and applying smote and split - Failed')
    error_logger.error('Cluster formation and applying smote and split - Failed')
    error_logger.error('Exception: ', exc_info=True)
    #input("press Enter to Close")
    sys.exit()                     


192.168.1.112 : 2024-09-05 15:44:51,920 : INFO : [3073169385.py:5] : Cluster formation and applying smote and split


In [6]:
#==================================================================================================
### Model Creation for Preferred_clothing ### 
#==================================================================================================
try:
    audit_logger.info('Model Creation for Preferred_clothing')
    df_preferred_clothing_train = df_resampled[['Gender', 'Weight_kg', 'Height_cm','Preferred_Clothing_Type', 'Clothing_Size',
                           'Favourite_Color_r', 'Favourite_Color_g', 'Favourite_Color_b','Best_Color_Confidence_r', 
                           'Best_Color_Confidence_g','Best_Color_Confidence_b', 'birthplace_lat','birthplace_lon', 'Cluster',
                           'resi_lat', 'resi_lon', 'Age', 'Preferred_Clothing', 'Personality']]
    df_preferred_clothing_train.drop_duplicates(inplace=True)
    X_train_1 = df_preferred_clothing_train[['Gender', 'Weight_kg', 'Height_cm','Preferred_Clothing_Type', 'Clothing_Size',
                           'Favourite_Color_r', 'Favourite_Color_g', 'Favourite_Color_b','Best_Color_Confidence_r', 
                           'Best_Color_Confidence_g','Best_Color_Confidence_b', 'birthplace_lat','birthplace_lon', 'Cluster',
                           'resi_lat', 'resi_lon', 'Age', 'Personality']]
    y_train_1 = df_preferred_clothing_train["Preferred_Clothing"]
    
    # Initialize and train RandomForest
    rf_model = RandomForestClassifier(criterion= 'gini', max_depth= None, min_samples_leaf= 1, min_samples_split= 2, n_estimators= 250 , random_state=42)
    rf_model.fit(X_train_1, y_train_1)
    
    # Initialize and train LightGBM
    lgb_model = lgb.LGBMClassifier(boosting_type='dart', learning_rate=0.8, max_depth=12, n_estimators=200, num_leaves=158, subsample=0.4)
    lgb_model.fit(X_train_1, y_train_1)
    
    # Save the RandomForest model
    dump(rf_model, os.path.join(saved_folder,'random_forest_model_stage_1.joblib'))
    
    # Save the LightGBM model
    dump(lgb_model, os.path.join(saved_folder,'lightgbm_model_stage_1.joblib'))
except Exception as err:
    audit_logger.info('Model Creation for Preferred_clothing - Failed')
    error_logger.error('Model Creation for Preferred_clothing - Failed')
    error_logger.error('Exception: ', exc_info=True)
    #input("press Enter to Close")
    sys.exit()

192.168.1.112 : 2024-09-05 15:44:57,463 : INFO : [4204166383.py:5] : Model Creation for Preferred_clothing
[LightGBM] [Info] Number of positive: 2287, number of negative: 9450
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001740 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2928
[LightGBM] [Info] Number of data points in the train set: 11737, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.194854 -> initscore=-1.418774
[LightGBM] [Info] Start training from score -1.418774
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [7]:
#==================================================================================================
### Classifier for two piece wear ### 
#==================================================================================================
try:
    audit_logger.info('Classifier for two piece wear')
    df_twopiece_train = df_resampled[df_resampled["Preferred_Clothing"] == 1]
    df_twopiece_train = df_twopiece_train[['Gender', 'Weight_kg', 'Height_cm', 'Preferred_Clothing_Type','Personality', 'Clothing_Size','Favourite_Color_r', 
              'Favourite_Color_g', 'Favourite_Color_b','Best_Color_Confidence_r', 'Best_Color_Confidence_g','Best_Color_Confidence_b', 
              'birthplace_lat', 'birthplace_lon', 'resi_lat', 'resi_lon', 'Age', 
                'Preferred_Bottoms', 'Bottoms_Material','Bottoms_Length', 'Bottoms_Fitting', 'Preferred_Upperwear',
           'Upperwear_Material', 'Upperwear_Length', 'Upperwear_Neckline','Upperwear_Sleeve_Type', 'Upperwear_Pattern']]
    df_twopiece_train.drop_duplicates(inplace=True)
    X_twopiece_train = df_twopiece_train[['Gender', 'Weight_kg', 'Height_cm', 'Preferred_Clothing_Type','Personality', 'Clothing_Size','Favourite_Color_r', 
              'Favourite_Color_g', 'Favourite_Color_b','Best_Color_Confidence_r', 'Best_Color_Confidence_g','Best_Color_Confidence_b', 
              'birthplace_lat', 'birthplace_lon', 'resi_lat', 'resi_lon', 'Age']]
    y_twopiece_train = df_twopiece_train[['Preferred_Bottoms', 'Bottoms_Material','Bottoms_Length', 'Bottoms_Fitting', 'Preferred_Upperwear',
           'Upperwear_Material', 'Upperwear_Length', 'Upperwear_Neckline','Upperwear_Sleeve_Type', 'Upperwear_Pattern']]
    
    # Initialize a Random Forest Classifier
    rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1, criterion='entropy', random_state=42)
    
    # Wrap it in the MultiOutputClassifier
    multi_rf_classifier = MultiOutputClassifier(rf_classifier)
    
    # Training the classifier
    multi_rf_classifier.fit(X_twopiece_train, y_twopiece_train)
    
    # Save the Multiclass RandomForest model
    dump(multi_rf_classifier, os.path.join(saved_folder,'multiclass_random_forest_2_stage_2.joblib'))
except Exception as err:
    audit_logger.info('Classifier for two piece wear - Failed')
    error_logger.error('Classifier for two piece wear - Failed')
    error_logger.error('Exception: ', exc_info=True)
    #input("press Enter to Close")
    sys.exit() 

192.168.1.112 : 2024-09-05 15:45:01,727 : INFO : [4201582880.py:5] : Classifier for two piece wear


In [8]:
#==================================================================================================
### Deeplearning Model for two piece wear ### 
#==================================================================================================
try:
    audit_logger.info('Deeplearning Model for two piece wear')
    df_color_twopiece_train = df_resampled[df_resampled["Preferred_Clothing"] == 1]
    df_color_twopiece_train = df_color_twopiece_train[['Gender', 'Weight_kg', 'Height_cm', 'Preferred_Clothing_Type','Personality', 'Clothing_Size','Favourite_Color_r', 
               'Favourite_Color_g', 'Favourite_Color_b','Best_Color_Confidence_r', 'Best_Color_Confidence_g','Best_Color_Confidence_b', 
               'birthplace_lat', 'birthplace_lon', 'resi_lat', 'resi_lon', 'Age', 
                'Preferred_Bottoms', 'Bottoms_Color_r', 'Bottoms_Color_g','Bottoms_Color_b', 'Preferred_Upperwear',
           'Upperwear_Color_r', 'Upperwear_Color_g', 'Upperwear_Color_b']]
    df_color_twopiece_train.drop_duplicates(inplace=True)
    X_color_twopiece_train = df_color_twopiece_train[['Gender', 'Weight_kg', 'Height_cm', 'Preferred_Clothing_Type','Personality', 'Clothing_Size','Favourite_Color_r', 
               'Favourite_Color_g', 'Favourite_Color_b','Best_Color_Confidence_r', 'Best_Color_Confidence_g','Best_Color_Confidence_b', 
               'birthplace_lat', 'birthplace_lon', 'resi_lat', 'resi_lon', 'Age']]
    y_color_twopiece_train = df_color_twopiece_train[['Preferred_Bottoms', 'Bottoms_Color_r', 'Bottoms_Color_g','Bottoms_Color_b', 'Preferred_Upperwear',
           'Upperwear_Color_r', 'Upperwear_Color_g', 'Upperwear_Color_b']]
    
    # Normalize features and targets
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X_color_twopiece_train)
    y_scaled = scaler_y.fit_transform(y_color_twopiece_train)
    
    dump(scaler_X, os.path.join(saved_folder,'scaler_X_twop.joblib')) #save the scaler
    dump(scaler_y, os.path.join(saved_folder,'scaler_y_twop.joblib')) #save the scaler
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
    
    # Reshape input to be [samples, features, 1] for 1D CNN
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    # Build 1D CNN model
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
        MaxPooling1D(pool_size=5),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    
    # Implement early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
    
    # Save the model
    model.save(os.path.join(saved_folder,'cnn_model_twopiece.h5'))
except Exception as err:
    audit_logger.info('Deeplearning Model for two piece wear - Failed')
    error_logger.error('Deeplearning Model for two piece wear - Failed')
    error_logger.error('Exception: ', exc_info=True)
    #input("press Enter to Close")
    sys.exit()    

192.168.1.112 : 2024-09-05 15:45:09,522 : INFO : [3050605927.py:5] : Deeplearning Model for two piece wear


2024-09-05 15:45:09.576369: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-05 15:45:09.576437: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2024-09-05 15:45:10.467506: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-09-05 15:45:10.536823: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


204/208 [============================>.] - ETA: 0s - loss: 0.1340 - mean_squared_error: 0.1340

...

191/208 [==========================>...] - ETA: 0s - loss: 0.1282 - mean_squared_error: 0.1282

196/208 [===========================>..] - ETA: 0s - loss: 0.1280 - mean_squared_error: 0.1280

201/208 [===========================>..] - ETA: 0s - loss: 0.1280 - mean_squared_error: 0.1280

206/208 [============================>.] - ETA: 0s - loss: 0.1279 - mean_squared_error: 0.1279

208/208 [==============================] - 2s 12ms/step - loss: 0.1279 - mean_squared_error: 0.1279 - val_loss: 0.1312 - val_mean_squared_error: 0.1312


In [9]:
#==================================================================================================
### Classifier for one piece wear ### 
#==================================================================================================
try:
    audit_logger.info('Classifier for one piece wear')
    df_onepiece_train = df_resampled[df_resampled["Preferred_Clothing"] == 0]
    df_onepiece_train = df_onepiece_train[['Gender', 'Weight_kg', 'Height_cm', 'Preferred_Clothing_Type','Personality', 'Clothing_Size','Favourite_Color_r', 
            'Favourite_Color_g', 'Favourite_Color_b','Best_Color_Confidence_r', 'Best_Color_Confidence_g','Best_Color_Confidence_b', 
            'birthplace_lat', 'birthplace_lon', 'resi_lat', 'resi_lon', 'Age', 'Favourite_Single_Piece', 'Single_Piece_Material', 
            'Single_Piece_Fitting', 'Single_Piece_Length','Single_Piece_Sleeves', 'Single_Piece_Neckline', 'Single_Piece_Pattern']]
    df_onepiece_train.drop_duplicates(inplace=True)
    X_onepiece_train = df_onepiece_train[['Gender', 'Weight_kg', 'Height_cm', 'Preferred_Clothing_Type','Personality', 'Clothing_Size','Favourite_Color_r', 
              'Favourite_Color_g', 'Favourite_Color_b','Best_Color_Confidence_r', 'Best_Color_Confidence_g','Best_Color_Confidence_b', 
              'birthplace_lat', 'birthplace_lon', 'resi_lat', 'resi_lon', 'Age']]
    y_onepiece_train = df_onepiece_train[['Favourite_Single_Piece', 'Single_Piece_Material', 'Single_Piece_Fitting', 'Single_Piece_Length',
              'Single_Piece_Sleeves', 'Single_Piece_Neckline', 'Single_Piece_Pattern']]
    
    # Initialize a Random Forest Classifier
    rf_classifier = RandomForestClassifier(criterion='entropy', max_depth=None, min_samples_leaf=1, min_samples_split= 2, n_estimators=200, random_state=42)
    
    # Wrap it in the MultiOutputClassifier
    multi_rf_classifier = MultiOutputClassifier(rf_classifier)
    
    # Training the classifier
    multi_rf_classifier.fit(X_onepiece_train, y_onepiece_train)
    
    # Save the Multiclass RandomForest model
    dump(multi_rf_classifier, os.path.join(saved_folder,'multiclass_random_forest_1_stage_2.joblib'))
except Exception as err:
    audit_logger.info('Classifier for one piece wear - Failed')
    error_logger.error('Classifier for one piece wear - Failed')
    error_logger.error('Exception: ', exc_info=True)
    #input("press Enter to Close")
    sys.exit() 

192.168.1.112 : 2024-09-05 15:46:30,217 : INFO : [1814071630.py:5] : Classifier for one piece wear


In [10]:
#==================================================================================================
### Deeplearning Model for two piece wear ### 
#==================================================================================================
try:
    audit_logger.info('Deeplearning Model for one piece wear')
    df_color_onepiece_train = df_resampled[df_resampled["Preferred_Clothing"] == 0]
    df_color_onepiece_train = df_color_onepiece_train[['Gender', 'Weight_kg', 'Height_cm', 'Preferred_Clothing_Type', 'Clothing_Size',
           'Favourite_Color_r', 'Favourite_Color_g', 'Favourite_Color_b','Best_Color_Confidence_r', 'Best_Color_Confidence_g',
            'Best_Color_Confidence_b', 'birthplace_lat', 'birthplace_lon', 'resi_lat', 'resi_lon', 'Age', 'Favourite_Single_Piece', 
            'Single_Piece_Color_r', 'Single_Piece_Color_g', 'Single_Piece_Color_b', 'Personality']]
    df_color_onepiece_train.drop_duplicates(inplace=True)
    X_color_onepiece_train = df_color_onepiece_train[['Gender', 'Weight_kg', 'Height_cm', 'Preferred_Clothing_Type', 'Clothing_Size',
           'Favourite_Color_r', 'Favourite_Color_g', 'Favourite_Color_b','Best_Color_Confidence_r', 'Best_Color_Confidence_g',
            'Best_Color_Confidence_b', 'birthplace_lat', 'birthplace_lon', 'resi_lat', 'resi_lon', 'Age','Personality']]
    y_color_onepiece_train = df_color_onepiece_train[['Favourite_Single_Piece', 'Single_Piece_Color_r', 'Single_Piece_Color_g', 
                                                      'Single_Piece_Color_b']]
    
    # Normalize features and targets
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X_color_onepiece_train)
    y_scaled = scaler_y.fit_transform(y_color_onepiece_train)
    
    dump(scaler_X, os.path.join(saved_folder,'scaler_X_onep.joblib')) #save the scaler
    dump(scaler_y, os.path.join(saved_folder,'scaler_y_onep.joblib')) #save the scaler
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
    
    # Reshape input to be [samples, features, 1] for 1D CNN
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    # Build 1D CNN model
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
        MaxPooling1D(pool_size=5),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    
    # Implement early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
    
    # Save the model
    model.save(os.path.join(saved_folder,'cnn_model_onepiece.h5'))
except Exception as err:
    audit_logger.info('Deeplearning Model for one piece wear - Failed')
    error_logger.error('Deeplearning Model for one piece wear - Failed')
    error_logger.error('Exception: ', exc_info=True)
    #input("press Enter to Close")
    sys.exit()    

192.168.1.112 : 2024-09-05 15:47:22,440 : INFO : [4094372514.py:5] : Deeplearning Model for one piece wear
Epoch 1/50


2024-09-05 15:47:23.218144: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-09-05 15:47:23.262438: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


241/241 [==============================] - ETA: 0s - loss: 0.0179 - mean_squared_error: 0.0179

2024-09-05 15:47:26.744992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


241/241 [==============================] - 4s 15ms/step - loss: 0.0179 - mean_squared_error: 0.0179 - val_loss: 0.0155 - val_mean_squared_error: 0.0155
Epoch 2/50
241/241 [==============================] - 3s 13ms/step - loss: 0.0162 - mean_squared_error: 0.0162 - val_loss: 0.0154 - val_mean_squared_error: 0.0154
Epoch 3/50
241/241 [==============================] - 3s 13ms/step - loss: 0.0158 - mean_squared_error: 0.0158 - val_loss: 0.0146 - val_mean_squared_error: 0.0146
Epoch 4/50
241/241 [==============================] - 3s 12ms/step - loss: 0.0157 - mean_squared_error: 0.0157 - val_loss: 0.0142 - val_mean_squared_error: 0.0142
Epoch 5/50
241/241 [==============================] - 3s 12ms/step - loss: 0.0156 - mean_squared_error: 0.0156 - val_loss: 0.0144 - val_mean_squared_error: 0.0144
Epoch 6/50
241/241 [==============================] - 3s 12ms/step - loss: 0.0153 - mean_squared_error: 0.0153 - val_loss: 0.0144 - val_mean_squared_error: 0.0144
Epoch 7/50
241/241 [=============

...

217/240 [==========================>...] - ETA: 0s - loss: 0.0171 - mean_squared_error: 0.0171

223/240 [==========================>...] - ETA: 0s - loss: 0.0171 - mean_squared_error: 0.0171

229/240 [===========================>..] - ETA: 0s - loss: 0.0170 - mean_squared_error: 0.0170

235/240 [============================>.] - ETA: 0s - loss: 0.0169 - mean_squared_error: 0.0169

240/240 [==============================] - 2s 10ms/step - loss: 0.0168 - mean_squared_error: 0.0168 - val_loss: 0.0167 - val_mean_squared_error: 0.0167


In [11]:
audit_logger.info('Process run success')
total_time = datetime.now() - curr_time
audit_logger.info(f'Total time taken = {total_time}')

192.168.1.112 : 2024-09-05 15:48:26,119 : INFO : [2214019538.py:1] : Process run success
192.168.1.112 : 2024-09-05 15:48:26,119 : INFO : [2214019538.py:3] : Total time taken = 0:03:34.220570
